<a href="https://colab.research.google.com/github/Clarkdrengen/AAI-site/blob/master/Coin_flips_SVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coin flip model with SVI

First, install the required Python packages on the fly on Colab.

In [ ]:
!pip3 install pyro-ppl
!pip3 install matplotlib
!pip3 install scipy
!pip3 install numpy

Import the required Python packages.

In [ ]:
import pyro
import torch
import numpy
import scipy
import matplotlib.pyplot as plt
from scipy import stats
import pyro.distributions as pdist
import torch.distributions as tdist
import matplotlib.pyplot as plt

Create some coin flip data (0=heads, 1=tails).

In [ ]:
# Coin flips
y=[0]*50+[1]*100
# Turn the data into PyTorch tensor
y=torch.tensor(y, dtype=torch.float)

Set some options for inference; number of SVI steps and frequency of reporting the ELBO.

In [ ]:
STEPS=5000
REPORT=500

The probabilistic model, implemented as a function.
Prior: [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) over p. 
Likelihood: [Bernoulli distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution) for outcome.

In [ ]:
def model(y=None, n=None):
  # Prior - Beta over probability of heads (0)
  p=pyro.sample("p", pdist.Beta(10,10))
  # Likelihood - Bernoulli
  with pyro.plate("flips", n):
    obs=pyro.sample("obs", pdist.Bernoulli(p), obs=y)

The guide (which will approximate the model), implemented as a function. The approximating distribution is a [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution). 

In [ ]:
def guide(y, n):
  # Parameters of Beta distribution - these will be estimated using SVI
  # Notice the starting value and the constraint
  alpha_q=pyro.param("alpha_q", torch.tensor(15), constraint=tdist.constraints.positive)
  beta_q=pyro.param("beta_q", torch.tensor(15), constraint=tdist.constraints.positive)
  pyro.sample("p", pdist.Beta(alpha_q, beta_q))

Perform SVI - optimize the ELBO.

In [ ]:
# Clear Pyro's parameter store
pyro.clear_param_store()

In [ ]:
# Optimizer
adam_params={"lr":0.01}
optimizer=pyro.optim.Adam(adam_params)

In [ ]:
# Stochastic variational inference
svi=pyro.infer.SVI(model, guide, optimizer, loss=pyro.infer.Trace_ELBO())

In [ ]:
elbo=[]
n=y.shape[0] # number of data points
for step in range(STEPS):
  loss=svi.step(y, n)
  if step%REPORT==0:
    print("[Iteration %04d] loss: %.4f" % (step, loss))
  elbo.append(loss)

ELBO plot, so we can investigate convergence. 

In [ ]:
# ELBO vs. iteration plot
plt.xlabel("Iteration")
plt.ylabel("ELBO")
plt.plot(elbo)

Get the optimized parameters of the guide. These are the parameters of the variational posterior (a Beta distributrion).

In [ ]:
alpha_q=pyro.param("alpha_q").item()
beta_q=pyro.param("beta_q").item()

Get the mean and the standard deviation of the resulting Beta distribution and calculate a (variational) [Bayesian credible interval](https://en.wikipedia.org/wiki/Credible_interval).

In [ ]:
# Mean and standard deviation of estimated Beta distribution
beta=scipy.stats.beta(alpha_q, beta_q)
m=beta.mean()
s=numpy.sqrt(beta.var())

# Report mean and standard deviation
print("Mean and standard deviation: %.2f +/- %.2f" % (m, s))

# Report (variational) Bayesian 94% credible interval
# ppf=Percent point function (ie. inverse of cdf) 
low, high=beta.ppf([0.03, 0.97])
print("Variational 94%% Bayesian credible interval: [%.2f, %.2f]" % (low, high))